In [ ]:
import sys
sys.path.insert(19, 'litepose-pose-estimation/src')
sys.path.insert(20, 'src/referee_gloves_detector')
import json
import torch
from torchvision import transforms
import lp_config.lp_common_config as cc
from lp_model.lp_litepose import LitePose
from lp_inference.lp_inference import inference, assocEmbedding
from lp_utils.lp_image_processing import drawHeatmap, drawKeypoints, drawSkeleton
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
from preprocessing import red_filtering, segmentation_and_cropping, equalizing, squaring
import math
from Homography import Homography

In [ ]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_big_arch = config_data['path_big_arch']
file_path_csv_keypoints = config_data['path_csv_keypoints']

In [ ]:
model = LitePose().to(cc.config["device"])
model.load_state_dict(torch.load(file_path_big_arch, map_location=cc.config["device"]))

In [ ]:
def keypoints_saver(timestamps, keypoints_vec):
    d = {'timestamp':[],'x': [], 'y': [], 'tag':[]}
    df_keypoints = pd.DataFrame(data=d)
    for timestamp, restricted_keypoints in zip(timestamps, keypoints_vec):
        for i in range(len(restricted_keypoints[0])):
            if not (restricted_keypoints[0][i]):
                df_temp = pd.DataFrame([{'timestamp': '01/01/1970 01:00:00','x':np.nan, 'y':np.nan, 'tag':np.nan}])
                df_keypoints = pd.concat([df_keypoints, df_temp])
            else:
                df_temp = pd.DataFrame(restricted_keypoints[0][i])
                df_temp['timestamp'] = timestamp
                df_keypoints = pd.concat([df_keypoints, df_temp])
    df_keypoints.to_csv(file_path_csv_keypoints, index=False)
    df_multiindex = df_keypoints.set_index(['timestamp'])
    #display(df_multiindex)

In [ ]:
# HOMOGRAPHY IMAGE

uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\resources\\bloisi_nao_red_hands.png"
img = Image.open(uri).convert('RGB')
resize = transforms.Resize([224, 224])
img = resize(img)
to_tensor = transforms.ToTensor()
tensor = to_tensor(img)
tensor = tensor.unsqueeze(0)
output, keypoints = inference(model, tensor)
embedding = assocEmbedding(keypoints)
idx = 0
img = drawKeypoints(tensor[idx], keypoints[idx])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
im_pil = Image.fromarray(img)
im_pil.save('src/referee_gloves_detector/resources/src.jpg')
keypoints_saver(['01/01/1970 01:00:00'], [keypoints])
src = cv2.imread("src/referee_gloves_detector/resources/src.jpg")
dst = cv2.imread("src/referee_gloves_detector/resources/dst.jpg")
h = Homography(src,dst)
h._from_detection()
plan_view = cv2.warpPerspective(src, h.H, (dst.shape[1], dst.shape[0]))
cv2.imshow("Reprojected view", plan_view)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# HOMOGRAPHY VIDEO

uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\resources\\referee_1.mp4"
cap = cv2.VideoCapture(uri)

# Verifico se il video è stato aperto correttamente
if not cap.isOpened():
    raise Exception("Errore nell'apertura del video")

keypoints_vec = []
timestamps = []
resize = transforms.Resize([224, 224])
to_tensor = transforms.ToTensor()
ret,frame=cap.read()
count = 0
while ret:

    istante_attuale = datetime.now()
    stringa_istante = istante_attuale.strftime("%d/%m/%Y %H:%M:%S.%f")
    timestamps.append(stringa_istante)

    #-----------------------------PREPROCESSING START HERE-----------------------------
    #NORMALIZATION
    normalized_image = cv2.normalize(frame, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    #EQUALIZATION
    equalized_image = equalizing(normalized_image)
    #------------------------------PREPROCESSING END HERE------------------------------
    img = cv2.cvtColor(equalized_image, cv2.COLOR_BGR2RGB)
    img = squaring(img)

    try:
        # Può dare problemi se gli arriva una immagine con una dimensione mancante,
        # non ho capito il perchè (probabilmente è causata dall'algoritmo di cropping)
        # ma questo può succedere
        im_pil = Image.fromarray(img)
    except:
        print("Shape dell'immagine che farebbe crashare il processo: " + str(img.shape))
        pass
    frame = resize(im_pil)
    tensor = to_tensor(frame)
    tensor = tensor.unsqueeze(0)

    if ret==True:
        output, keypoints = inference(model, tensor)
        keypoints_vec.append(keypoints)
        embedding = assocEmbedding(keypoints)
        #frame_modified = drawSkeleton(tensor[0], embedding[0])
        frame = drawKeypoints(tensor[0], keypoints[0])
        if count == 0:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image_pil = Image.fromarray(frame, 'RGB')
            image_pil.save('src/referee_gloves_detector/resources/src.jpg')
            keypoints_saver(timestamps, keypoints_vec)
            src = cv2.imread("src/referee_gloves_detector/resources/src.jpg")
            dst = cv2.imread("src/referee_gloves_detector/resources/dst.jpg")
            h = Homography(src,dst)
            h._from_detection()
            count = count + 1
        frame = cv2.warpPerspective(frame, h.H, (dst.shape[1], dst.shape[0]))
        cv2.imshow("Pose estimation", frame)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break

    ret, frame = cap.read()
cap.release()
cv2.destroyAllWindows()
print("Dimensioni dell'array di frame:", len(keypoints_vec))
print("Dimensioni dell'array di timestamp:", len(timestamps))

In [ ]:
# Apro la webcam e salvo al volo i frame e il timestamp

webcam=cv2.VideoCapture(0)

if not webcam.isOpened():
    raise Exception("Errore nell'apertura della webcam")

keypoints_vec = []
timestamps = []
resize = transforms.Resize([224, 224])
to_tensor = transforms.ToTensor()
ret,frame=webcam.read()
while ret:


    istante_attuale = datetime.now()
    stringa_istante = istante_attuale.strftime("%d/%m/%Y %H:%M:%S.%f")
    timestamps.append(stringa_istante)

    frame = cv2.flip(frame, 1)
    #-----------------------------PREPROCESSING START HERE-----------------------------
    #RED FILTERING
    full_mask = red_filtering(frame)
    #SEGMENTATION E CROPPING
    cropped_image = segmentation_and_cropping(frame, full_mask)
    #NORMALIZATION
    normalized_image = cv2.normalize(cropped_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    #EQUALIZATION
    equalized_image = equalizing(normalized_image)
    #------------------------------PREPROCESSING END HERE------------------------------
    img = cv2.cvtColor(equalized_image, cv2.COLOR_BGR2RGB)
    img = squaring(img)

    try:
        # Può dare problemi se gli arriva una immagine con una dimensione mancante,
        # non ho capito il perchè (probabilmente è causata dall'algoritmo di cropping)
        # ma questo può succedere
        im_pil = Image.fromarray(img)
    except:
        print("Shape dell'immagine che farebbe crashare il processo: " + str(img.shape))
        pass
    frame = resize(im_pil)
    tensor = to_tensor(frame)
    tensor = tensor.unsqueeze(0)

    if ret==True:
        output, keypoints = inference(model, tensor)
        keypoints_vec.append(keypoints)
        embedding = assocEmbedding(keypoints)
        frame_modified = drawSkeleton(tensor[0], embedding[0])
        #frame = drawKeypoints(tensor[0], keypoints[0])
        cv2.imshow("Pose estimation", frame_modified)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break
    ret, frame = webcam.read()
webcam.release()
cv2.destroyAllWindows()
print("Dimensioni dell'array di frame:", len(keypoints_vec))
print("Dimensioni dell'array di timestamp:", len(timestamps))